# 利用 LOCC 来进行两方量子态分辨


<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概述

量子态分辨（quantum state discrimination, QSD）[1-2] 是量子通信，量子计算和量子密码学中的一个基本问题。本教程展示了如何通过本地量子操作和经典通信（LOCC）来区分满足 $\langle\psi\lvert\phi\rangle=0$ 的两个正交的两方纯态 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$。本教程中使用的方法和理论参考了 [3] 。


## 寻找一个态分辨协议

首先，我们将需要处理的问题定义清楚。考虑两个在空间上相隔一定距离的参与方 $ A $（Alice）和 $ B $（Bob）共享一对两个量子比特系统，该系统的状态 $|\varphi\rangle$ 由第三方 $C$（Charlie）提前制备好分发给参与方。Alice 和 Bob 只是被提前通知了 $|\varphi\rangle$ 可能是 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$ 两者其中之一，并且满足条件 $\langle\psi\lvert\phi\rangle=0$。然后，Charlie 给参与方提供了很多 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$ 的拷贝并希望 Alice 和 Bob 可以通过这些来正确的分辨出他们现在持有的量子态究竟是 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$ 中的哪一个。

在我们提供的 LOCCNet 框架下解决这样一个任务是很轻松的。方便起见，我们还是先采用单轮通讯协议 $r=1$ 的设计。 也就是说 Alice 和 Bob 之间只会进行一次经典通讯。具体的量子神经网络（QNN）见图 1。该任务中的关键步骤是确定损失函数 $ L $。这里我们设置 Alice 和 Bob 都需要对手中量子比特做一次测量，因此有 4 种可能的测量结果 $m_Am_B\in\{00, 01, 10, 11\}$。为了能清晰地区分 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$， 我们不妨定义当测量结果为 $m_Am_B\in\{00, 10\}$ 时将其分类为 $\lvert\psi\rangle$。 类似地，如果参与方的测量结果为 $m_Am_B\in\{01, 11\}$ 时将其分类为 $\lvert\phi\rangle$。此步骤可以理解为在监督学习中为数据添加标签。通过以上标签，我们可以通过分辨协议出错的概率来定义损失函数，

$$
L = p_{\lvert\psi\rangle\_01}+p_{\lvert\psi\rangle\_11}+p_{\lvert\phi\rangle\_10}+p_{\lvert\phi\rangle\_00},
\tag{1}
$$

其中 $p_{\lvert\psi\rangle\_01}$ 代表输入态为 $\lvert\psi\rangle$ 时测量 01 的概率（**注释：** 这是一种分类错误的情况）。接着我们可以通过优化算法来最小化损失函数。

![](figures/discrimination-fig-circuit.png "图 1. 用 LOCCNet 进行两方态分辨的协议示意图")
<div style="text-align:center">图 1. 用 LOCCNet 进行两方态分辨的协议示意图 </div>

我们将整个流程进行如下总结：

1. Alice 和 Bob 各自拥有一个二量子比特系统的一部分，整个系统的量子态可能为 $\lvert\psi\rangle$ 或者 $\lvert\phi\rangle$。这两个态正交且均是纯态。
2. Alice 对自己的量子比特进行酉变换 $U_A$（单比特通用门）。
3. Alice 在计算基（computational basis）上测量其量子比特，测量结果 $m_A\in \{0, 1\}$。然后，她通过经典信道告知 Bob 自己的测量结果。
4. Bob 根据 Alice 的测量结果在自己量子比特上施加不同的门。如果 $ m_A = 0 $， Bob 对其持有的量子比特作用 $ U_{B0} $；如果 $ m_A = 1 $，则 Bob 施加 $ U_{B1} $。然后 Bob 测量自己的量子比特得到结果 $m_B \in \{0,1\}$。**注意**：这里 $ U_{B0} $ 和 $ U_{B1} $ 都是单比特上的广义旋转门。
5. 计算损失函数 $L = p_{\lvert\psi\rangle\_01}+p_{\lvert\psi\rangle\_11}+ p_{\lvert\phi\rangle\_10}+ p_{\lvert\phi\rangle\_00}$，并使用基于梯度的优化方法。
6. 重复 1-5，直到损失函数收敛。
7. Charlie 在 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$ 两个之间随机选一个，并检验 Alice 和 Bob 的分辨方案是否可以正确地进行分辨。




## Paddle Quantum 代码实现

首先，我们导入相关的依赖包。

In [2]:
import numpy as np
from scipy.stats import unitary_group
import paddle
import paddle_quantum
from paddle_quantum.locc import LoccNet
# 切换至密度矩阵模式
paddle_quantum.set_backend('density_matrix')

Charlie 需要随机生成两个正交态 $\lvert\psi\rangle$ 以及 $\lvert\phi\rangle$。

In [3]:
def states_orthogonal_random(n, num=2):
    # 随机生成两个正交态
    assert num <= 2 ** n, "return too many orthognal states"
    U = unitary_group.rvs(2 ** n)
    return_list = [np.array(U[i], dtype=np.complex64) for i in range(num)]

    return return_list

下面是我们代码的主要部分，它定义了 Alice 和 Bob 的本地量子操作和损失函数。

In [4]:
class Net(LoccNet):
    def __init__(self):
        super(Net, self).__init__()
        # 添加第一个参与方 Alice
        # 第一个参数 1 代表着 Alice 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(1, party_name='Alice')
        # 添加第二个参与方 Bob
        # 第一个参数 1 代表着 Bob 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(1, party_name='Bob')

        # 将输入态写成密度矩阵形式
        _states = states_orthogonal_random(2)
        _states = [paddle_quantum.State(np.outer(init_state, init_state.conjugate())) for init_state in _states]
        # 初始化整个量子系统并分配量子态
        self.set_init_state(_states[0], [('Alice', 0), ('Bob', 0)])
        self.psi = self.init_status
        self.phi = self.reset_state(self.init_status, _states[1], [('Alice', 0), ('Bob', 0)])

        # Alice 的电路
        self.cirA = self.create_ansatz('Alice')
        # 添加单量子比特通用门
        self.cirA.u3(0)
        # Bob 要准备两个电路来应对两个不同的测量结果
        self.cirB = [self.create_ansatz('Bob'), self.create_ansatz('Bob')]
        # 添加单量子比特通用门
        self.cirB[0].u3(0)
        self.cirB[1].u3(0)

    def run_circuit(self, party, cir, state, res):
        # 运行电路
        after_state = cir(state)
        # 测量电路，记录结果 
        after_state = self.measure(status=after_state, which_qubits=(party, 0), results_desired=res)

        return after_state

    def forward(self):
        # 训练过程
        # Alice 操作过后的量子态
        psi = self.run_circuit('Alice', self.cirA, self.psi, ['0', '1'])
        phi = self.run_circuit('Alice', self.cirA, self.phi, ['0', '1'])

        # 定义损失函数
        loss = 0
        for each_psi in psi:
            if each_psi.measured_result == '0':
                psi_01 = self.run_circuit('Bob', self.cirB[0], each_psi, '1')
                loss += psi_01.prob
            elif each_psi.measured_result == '1':
                psi_11 = self.run_circuit('Bob', self.cirB[1], each_psi, '1')
                loss += psi_11.prob
        for each_phi in phi:
            if each_phi.measured_result == '0':
                phi_00 = self.run_circuit('Bob', self.cirB[0], each_phi, '0')
                loss += phi_00.prob
            elif each_phi.measured_result == '1':
                phi_10 = self.run_circuit('Bob', self.cirB[1], each_phi, '0')
                loss += phi_10.prob

        return loss

    def evaluate(self):
        # 测试过程
        choice = np.random.choice(['phi', 'psi'])
        if choice == 'phi':
            self.status = self.phi
        else:
            self.status = self.psi
        print('Charlie 选择的态是', choice)

        # Alice 的操作
        status = self.run_circuit('Alice', self.cirA, self.status, ['0', '1'])
        # Bob 的操作 
        result_0 = list()
        result_1 = list()
        for each_status in status:
            if each_status.measured_result == '0':
                status = self.run_circuit('Bob', self.cirB[0], each_status, ['0', '1'])
                result_0.append(status[0].prob.numpy()[0])
                result_0.append(status[1].prob.numpy()[0])
            elif each_status.measured_result == '1':
                status = self.run_circuit('Bob', self.cirB[1], each_status, ['0', '1'])
                result_1.append(status[0].prob.numpy()[0])
                result_1.append(status[1].prob.numpy()[0])

        print("Alice 和 Bob 将这个态分辨为 psi 的概率为:", result_0[0] + result_1[0])
        print("Alice 和 Bob 将这个态分辨为 phi 的概率为:", result_0[1] + result_1[1])

训练 Alice 和 Bob 的电路，然后随机选择两个正交态 $\lvert\psi\rangle$ 和 $\lvert\phi\rangle$ 之一以通过我们训练的电路，以检查它们是否可以区分。

In [6]:
ITR = 100  # 设置训练步数
LR = 0.1   # 设置学习速率
SEED = 999 # 固定 PQC 中参数的随机种子
np.random.seed(SEED)
paddle.seed(SEED)

net = Net()
params = net.cirA.parameters() + net.cirB[0].parameters() + net.cirB[1].parameters()
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=params)
# 通过梯度下降训练 LOCC 网络以进行 ITR 次迭代
for itr in range(ITR):
    loss = net()
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()
    if itr % 10 == 0:
        print("itr " + str(itr) + ":", loss.numpy()[0])
print("最小损失:", loss.numpy()[0])

print("======================== 测试阶段 ===============================")
np.random.seed(10)
net.evaluate()
np.random.seed(6)
net.evaluate()

itr 0: 1.1238832
itr 10: 0.32665575
itr 20: 0.085007355
itr 30: 0.085270524
itr 40: 0.026622297
itr 50: 0.015240545
itr 60: 0.007836903
itr 70: 0.004827206
itr 80: 0.0035075857
itr 90: 0.002215183
最小损失: 0.0016813411
======================== 测试阶段 ===============================
Charlie 选择的态是 psi
Alice 和 Bob 将这个态分辨为 psi 的概率为: 0.9990063
Alice 和 Bob 将这个态分辨为 phi 的概率为: 0.0009937042
Charlie 选择的态是 phi
Alice 和 Bob 将这个态分辨为 psi 的概率为: 0.0006236615
Alice 和 Bob 将这个态分辨为 phi 的概率为: 0.9993763


## 结论

从模拟结果可以看出，经过训练的量子电路可以准确地分辨出两个正交的量子态，精确度 $>99.9\%$。这里有一个值得思考的问题：我们是否可以通过添加更多态来推广这种分辨方案？

---
## 参考文献

[1] Barnett, Stephen M., and Sarah Croke. "Quantum state discrimination." [Advances in Optics and Photonics 1.2 (2009): 238-278.](https://www.osapublishing.org/abstract.cfm?id=176580)

[2] Chefles, Anthony. "Quantum state discrimination." [Contemporary Physics 41.6 (2000): 401-424.](https://arxiv.org/abs/quant-ph/0010114)

[3] Walgate, Jonathan, et al. "Local distinguishability of multipartite orthogonal quantum states." [Physical Review Letters 85.23 (2000): 4972.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.85.4972)
